# Part 1 - Extracting and Saving Data from Yelp API

## Obective

- For this CodeAlong, we will be working with the Yelp API. 
- You will use the the Yelp API to search your home town for a cuisine type of your choice.
- Next class, we will then use Plotly Express to create a map with the Mapbox API to visualize the results.
    
    

## Tools You Will Use
- Part 1:
    - Yelp API:
        - Getting Started: 
            - https://www.yelp.com/developers/documentation/v3/get_started

    - `YelpAPI` python package
        -  "YelpAPI": https://github.com/gfairchild/yelpapi
- Part 2:

    - Plotly Express: https://plotly.com/python/getting-started/
        - With Mapbox API: https://www.mapbox.com/
        - `px.scatter_mapbox` [Documentation](https://plotly.com/python/scattermapbox/): 




### Applying Code From
- Efficient API Calls Lesson Link: https://login.codingdojo.com/m/376/12529/88078

In [1]:
!pip install tqdm

In [2]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

## 1. Registering for Required APIs


- Yelp: https://www.yelp.com/developers/documentation/v3/get_started


> Check the official API documentation to know what arguments we can search for: https://www.yelp.com/developers/documentation/v3/business_search

### Load Credentials and Create Yelp API Object

In [3]:
# Load API Credentials
import json
with open('/Users/ashley/.secret/yelp_api.json') as f:
    login = json.load(f)
login.keys()

dict_keys(['client-id', 'api-key'])

In [4]:
# Instantiate YelpAPI Variable
from yelpapi import YelpAPI
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)
yelp_api

### Define Search Terms and File Paths

In [5]:
# set our API call parameters and filename before the first call
LOCATION = "Philadelphia, PA 19145"
TERM = "Vegan"

In [6]:
## Specify fodler for saving data
FOLDER = "Data/" 
os.makedirs(FOLDER,exist_ok = True)
# Specifying JSON_FILE filename (can include a folder)
JSON_FILE = FOLDER+ f"{LOCATION.split(',')[0]}-{TERM}.json"

### Check if Json File exists and Create it if it doesn't

In [7]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f"[i] {JSON_FILE} not found. Saving empty list to file.")
    
    
    ## save the first page of results
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
## If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/Philadelphia-Vegan.json not found. Saving empty list to file.


### Load JSON FIle and account for previous results

In [8]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


### Make the first API call to get the first page of data

- We will use this first result to check:
    - how many total results there are?
    - Where is the actual data we want to save?
    - how many results do we get at a time?


In [9]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [10]:
## How many results total?
total_results = results['total']
total_results

1300

- Where is the actual data we want to save?

In [11]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

- Calculate how many pages of results needed to cover the total_results

In [12]:
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

65

In [13]:
    ## The block of code we want to TRY to run
    for i in tqdm_notebook( range(1,n_pages+1)):
        try:
            time.sleep(.2)    

            ## Read in results in progress file and check the length
            with open(JSON_FILE, 'r') as f:
                prev_results =json.load(f)

            ## save number of results for to use as offset
            n_results = len(prev_results)


            ## use n_results as the OFFSET 
            results = yelp_api.search_query(term = TERM, location=LOCATION, 
                                            offset=n_results)

            ## append new results and save to file
            prev_results.extend(results['businesses'])
            with open(JSON_FILE, 'w') as f:
                json.dump(prev_results,f)
    ## What to do if we get an error/exception.
        except Exception as e:
            print('[!] ERROR: ', e)
            break
print('Loop completed.')


  0%|          | 0/65 [00:00<?, ?it/s]

[!] ERROR:  VALIDATION_ERROR: Too many results requested, limit+offset must be <= 1000.
Loop completed.


## Open the Final JSON File with Pandas

In [14]:
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,QSmuCHi63m8zEJvUc6SDAA,monster-vegan-philadelphia,Monster Vegan,https://s3-media1.fl.yelpcdn.com/bphoto/KJl1P_...,False,https://www.yelp.com/biz/monster-vegan-philade...,96,"[{'alias': 'vegan', 'title': 'Vegan'}, {'alias...",4.5,"{'latitude': 39.94676119356413, 'longitude': -...","[pickup, delivery]",$$,"{'address1': '1229 Spruce St', 'address2': '',...",+12157909494,(215) 790-9494,4356.807519
1,u-xGrOvBv1bxtpAHk-SZ5w,vegan-commissary-philadelphia-2,Vegan Commissary,https://s3-media3.fl.yelpcdn.com/bphoto/_50LQX...,False,https://www.yelp.com/biz/vegan-commissary-phil...,145,"[{'alias': 'bakeries', 'title': 'Bakeries'}, {...",4.5,"{'latitude': 39.9222106933594, 'longitude': -7...",[delivery],$$,"{'address1': '1429 Wolf St', 'address2': None,...",+12159643232,(215) 964-3232,1939.570148
2,TyhguONKc8i6M7wP09nTbA,hibiscus-cafe-philadelphia,Hibiscus Cafe,https://s3-media1.fl.yelpcdn.com/bphoto/kD_dVv...,False,https://www.yelp.com/biz/hibiscus-cafe-philade...,157,"[{'alias': 'vegetarian', 'title': 'Vegetarian'...",4.5,"{'latitude': 39.94851, 'longitude': -75.22156}","[pickup, delivery]",$$,"{'address1': '4907 Catharine St', 'address2': ...",+12153073749,(215) 307-3749,4482.689781
3,2CDI713ATuxHfnB5b-sBdw,vedge-philadelphia,Vedge,https://s3-media3.fl.yelpcdn.com/bphoto/fU-wK0...,False,https://www.yelp.com/biz/vedge-philadelphia?ad...,1484,"[{'alias': 'vegan', 'title': 'Vegan'}, {'alias...",4.5,"{'latitude': 39.9479310476302, 'longitude': -7...",[delivery],$$$,"{'address1': '1221 Locust St', 'address2': '',...",+12153207500,(215) 320-7500,4502.846821
4,5oFcqABGb-y38mB83f7NFg,mish-mish-philadelphia,Mish Mish,https://s3-media4.fl.yelpcdn.com/bphoto/zq3leR...,False,https://www.yelp.com/biz/mish-mish-philadelphi...,14,"[{'alias': 'mediterranean', 'title': 'Mediterr...",4.5,"{'latitude': 39.9296, 'longitude': -75.16305}",[delivery],NaN,"{'address1': '1046 Tasker St', 'address2': Non...",+12677619750,(267) 761-9750,2971.513807


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,3BJxm-HnvzdwD1zjmSbmyQ,golden-chopstick-chinese-restaurant-philadelphia,Golden Chopstick Chinese Restaurant,https://s3-media2.fl.yelpcdn.com/bphoto/ztDB9n...,False,https://www.yelp.com/biz/golden-chopstick-chin...,142,"[{'alias': 'chinese', 'title': 'Chinese'}]",3.0,"{'latitude': 39.9623477, 'longitude': -75.1624...","[pickup, delivery]",$,"{'address1': '490 N Broad St', 'address2': 'St...",+12154969190,(215) 496-9190,5843.628459
996,zf10JmXNZFljGl3IgWW-Lg,mizu-philadelphia,Mizu,https://s3-media2.fl.yelpcdn.com/bphoto/FnJ99I...,False,https://www.yelp.com/biz/mizu-philadelphia?adj...,122,"[{'alias': 'sushi', 'title': 'Sushi Bars'}]",3.5,"{'latitude': 39.9552752, 'longitude': -75.2022...","[pickup, delivery]",$$,"{'address1': '111 S 40th St', 'address2': '', ...",+12153821745,(215) 382-1745,4561.134310
997,jAKvrLDkkKai0shp-IY0pg,sugar-factory-philadelphia-philadelphia-3,Sugar Factory - Philadelphia,https://s3-media1.fl.yelpcdn.com/bphoto/NT-fkl...,False,https://www.yelp.com/biz/sugar-factory-philade...,72,"[{'alias': 'tradamerican', 'title': 'American ...",2.5,"{'latitude': 39.95041407925572, 'longitude': -...",[],NaN,"{'address1': '1216 Chestnut St', 'address2': '...",+12159774993,(215) 977-4993,4749.033521
998,n5Fdf9RzooS7obq0NecOWw,four-seasons-juice-bar-philadelphia,Four Seasons Juice Bar,https://s3-media4.fl.yelpcdn.com/bphoto/Vy_0yZ...,False,https://www.yelp.com/biz/four-seasons-juice-ba...,116,"[{'alias': 'juicebars', 'title': 'Juice Bars &...",4.5,"{'latitude': 39.9530036997233, 'longitude': -7...",[delivery],$,"{'address1': '51 N 12th St', 'address2': '', '...",,,5068.505876
999,AJchGMnrpfIbTULXE2fasA,baby-buns-philly-philadelphia,Baby Buns Philly,https://s3-media1.fl.yelpcdn.com/bphoto/9n8DnL...,False,https://www.yelp.com/biz/baby-buns-philly-phil...,23,"[{'alias': 'burgers', 'title': 'Burgers'}]",3.5,"{'latitude': 39.949747, 'longitude': -75.148196}","[pickup, delivery]",NaN,"{'address1': '111 S Independence Mall E', 'add...",+16092882721,(609) 288-2721,5391.831603


In [18]:
## convert the filename to a .csv.gz
csv_file = JSON_FILE.replace('.json','.csv.gz')
csv_file

'Data/Philadelphia-Vegan.csv.gz'

In [20]:
# save the final results to a compressed csv
final_df.to_csv(csv_file, 
                compression='gzip',index=False)

## Bonus: compare filesize with os module's `os.path.getsize`

In [21]:
size_json = os.path.getsize(JSON_FILE)
size_csv_gz = os.path.getsize(JSON_FILE.replace('.json','.csv.gz'))

print(f'JSON FILE: {size_json:,} Bytes')
print(f'CSV.GZ FILE: {size_csv_gz:,} Bytes')

print(f'the csv.gz is {size_json/size_csv_gz} times smaller!')

JSON FILE: 987,306 Bytes
CSV.GZ FILE: 141,596 Bytes
the csv.gz is 6.9726969688409275 times smaller!


## Next Class: Processing the Results and Mapping 